In [2]:
import bs4 as bs

import requests
import urllib.request
import pandas as pd
print('done')

done


In [3]:
from bs4 import BeautifulSoup

## Remake the df from part 1

In [4]:
#works better!
#read webpage into bs
source = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()
soup = bs.BeautifulSoup(source,'lxml')
#find the table
table = soup.find('table', attrs={'class':'wikitable sortable'})
table_rows = table.find_all('tr')
#loop over table rows and see what I get
res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)


torontoRaw = pd.DataFrame(res, columns=["Postcode", "Boro", "Neighborhood"])
#drop 'not assigned' boro
torontoBoro = torontoRaw[torontoRaw.Boro != 'Not assigned']
torontoBoro.Neighborhood.replace('Not assigned',torontoBoro.Boro,inplace=True)
toronto = pd.DataFrame(torontoBoro.groupby(['Postcode','Boro'])['Neighborhood'].apply(', '.join))
toronto = toronto.reset_index()
toronto.head(20)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pandas/core/generic.py:4619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


,Postcode,Boro,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


## Get Lat & Long from the csv file

In [8]:
url="https://cocl.us/Geospatial_data"
c=pd.read_csv(url)
c.rename(columns={'Postal Code': 'Postcode'}, inplace=True)
c.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Join lat/long to Toronto table using merge()

In [28]:
toronto_ll = pd.merge(toronto, c, on='Postcode', how='left')
toronto_ll.sort_values(['Neighborhood'])
toronto_ll.head(40)

,Postcode,Boro,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


## Done part 2